# 🎯 Lecture 12: Multi-Modal Data Integration - Hands-on Practice

## Table of Contents
1. [Early vs Late Fusion Implementation](#practice-1-early-vs-late-fusion)
2. [Matrix Factorization for Multi-Omics](#practice-2-matrix-factorization)
3. [Canonical Correlation Analysis (CCA)](#practice-3-canonical-correlation-analysis)
4. [Similarity Network Fusion](#practice-4-similarity-network-fusion)
5. [Attention-based Integration](#practice-5-attention-based-integration)
6. [MOFA (Multi-Omics Factor Analysis)](#practice-6-mofa-analysis)
7. [Integration Workflow](#practice-7-complete-integration-workflow)
8. [Visualization and Interpretation](#practice-8-visualization-and-interpretation)

## Installing and Importing Essential Libraries

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import NMF, PCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')
sns.set_palette('husl')

print("✅ All libraries loaded successfully!")

---
## Practice 1: Early vs Late Fusion

### 🎯 Learning Objectives
- Understand the difference between early and late fusion strategies
- Implement both approaches on multi-modal data
- Compare their performance and characteristics

### 📖 Key Concepts
- **Early Fusion**: Concatenate features before modeling (captures feature interactions)
- **Late Fusion**: Train separate models and combine predictions (more flexible)

In [ ]:
# 1.1 Generate synthetic multi-modal data
def generate_multimodal_data(n_samples=200, n_features_mod1=10, n_features_mod2=15):
    """Generate synthetic data for two modalities"""
    np.random.seed(42)
    
    # Modality 1: Genomics (e.g., gene expression)
    X_genomics = np.random.randn(n_samples, n_features_mod1)
    
    # Modality 2: Imaging (e.g., radiomics features)
    X_imaging = np.random.randn(n_samples, n_features_mod2)
    
    # Generate labels (binary classification: disease vs healthy)
    # Create dependency on both modalities
    signal_genomics = X_genomics[:, 0] + X_genomics[:, 1]
    signal_imaging = X_imaging[:, 0] - X_imaging[:, 1]
    y = (signal_genomics + signal_imaging > 0).astype(int)
    
    print("📊 Multi-modal Data Generated")
    print(f"  Genomics shape: {X_genomics.shape}")
    print(f"  Imaging shape: {X_imaging.shape}")
    print(f"  Labels distribution: {np.bincount(y)}")
    
    return X_genomics, X_imaging, y

X_gen, X_img, y = generate_multimodal_data()

In [ ]:
# 1.2 Early Fusion Implementation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

def early_fusion(X_mod1, X_mod2, y):
    """Implement early fusion strategy"""
    
    # Step 1: Concatenate features
    X_early = np.concatenate([X_mod1, X_mod2], axis=1)
    print("\n🔗 Early Fusion")
    print(f"  Combined feature shape: {X_early.shape}")
    
    # Step 2: Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_early, y, test_size=0.3, random_state=42
    )
    
    # Step 3: Train single model
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_scaled, y_train)
    
    # Step 4: Evaluate
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"\n  📈 Performance:")
    print(f"    Accuracy: {accuracy:.4f}")
    print(f"    AUC-ROC: {auc:.4f}")
    
    return model, accuracy, auc

early_model, early_acc, early_auc = early_fusion(X_gen, X_img, y)

In [ ]:
# 1.3 Late Fusion Implementation
def late_fusion(X_mod1, X_mod2, y):
    """Implement late fusion strategy"""
    
    print("\n🔀 Late Fusion")
    
    # Split data for both modalities
    X1_train, X1_test, y_train, y_test = train_test_split(
        X_mod1, y, test_size=0.3, random_state=42
    )
    X2_train, X2_test, _, _ = train_test_split(
        X_mod2, y, test_size=0.3, random_state=42
    )
    
    # Train separate models
    # Model 1: Genomics
    scaler1 = StandardScaler()
    X1_train_scaled = scaler1.fit_transform(X1_train)
    X1_test_scaled = scaler1.transform(X1_test)
    
    model1 = LogisticRegression(random_state=42, max_iter=1000)
    model1.fit(X1_train_scaled, y_train)
    pred_proba1 = model1.predict_proba(X1_test_scaled)[:, 1]
    
    # Model 2: Imaging
    scaler2 = StandardScaler()
    X2_train_scaled = scaler2.fit_transform(X2_train)
    X2_test_scaled = scaler2.transform(X2_test)
    
    model2 = LogisticRegression(random_state=42, max_iter=1000)
    model2.fit(X2_train_scaled, y_train)
    pred_proba2 = model2.predict_proba(X2_test_scaled)[:, 1]
    
    # Combine predictions (average ensemble)
    pred_proba_combined = (pred_proba1 + pred_proba2) / 2
    y_pred_combined = (pred_proba_combined > 0.5).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred_combined)
    auc = roc_auc_score(y_test, pred_proba_combined)
    
    print(f"\n  📈 Performance:")
    print(f"    Accuracy: {accuracy:.4f}")
    print(f"    AUC-ROC: {auc:.4f}")
    
    return model1, model2, accuracy, auc

late_model1, late_model2, late_acc, late_auc = late_fusion(X_gen, X_img, y)

In [ ]:
# 1.4 Compare Early vs Late Fusion
def compare_fusion_strategies():
    """Visualize comparison between fusion strategies"""
    
    results = pd.DataFrame({
        'Strategy': ['Early Fusion', 'Late Fusion'],
        'Accuracy': [early_acc, late_acc],
        'AUC-ROC': [early_auc, late_auc]
    })
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Accuracy comparison
    axes[0].bar(results['Strategy'], results['Accuracy'], color=['#1E64C8', '#5088d4'])
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title('Accuracy Comparison')
    axes[0].set_ylim([0.5, 1.0])
    for i, v in enumerate(results['Accuracy']):
        axes[0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')
    
    # AUC comparison
    axes[1].bar(results['Strategy'], results['AUC-ROC'], color=['#1E64C8', '#5088d4'])
    axes[1].set_ylabel('AUC-ROC')
    axes[1].set_title('AUC-ROC Comparison')
    axes[1].set_ylim([0.5, 1.0])
    for i, v in enumerate(results['AUC-ROC']):
        axes[1].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Comparison Summary:")
    print(results.to_string(index=False))
    print("\n✅ Key Insights:")
    print("  • Early fusion captures feature interactions but is computationally expensive")
    print("  • Late fusion is more flexible and modular")
    print("  • Performance depends on data characteristics and signal distribution")

compare_fusion_strategies()

---
## Practice 2: Matrix Factorization for Multi-Omics

### 🎯 Learning Objectives
- Implement Non-negative Matrix Factorization (NMF) for multi-omics data
- Learn to extract shared and specific factors
- Interpret biological meaning of factors

### 📖 Key Concepts
**Matrix Factorization**: $X \approx W \times H$ where $W$ contains sample factors and $H$ contains feature factors

In [ ]:
# 2.1 Generate multi-omics data
def generate_omics_data(n_samples=100, n_genes=50, n_proteins=40):
    """Generate synthetic omics data"""
    np.random.seed(42)
    
    # Create latent factors
    n_factors = 5
    W = np.abs(np.random.randn(n_samples, n_factors))  # Sample factors
    
    # Generate gene expression (transcriptomics)
    H_genes = np.abs(np.random.randn(n_factors, n_genes))
    X_genes = W @ H_genes + np.abs(np.random.randn(n_samples, n_genes) * 0.1)
    
    # Generate protein abundance (proteomics)
    H_proteins = np.abs(np.random.randn(n_factors, n_proteins))
    X_proteins = W @ H_proteins + np.abs(np.random.randn(n_samples, n_proteins) * 0.1)
    
    print("🧬 Multi-omics Data Generated")
    print(f"  Gene expression: {X_genes.shape}")
    print(f"  Protein abundance: {X_proteins.shape}")
    print(f"  True factors: {n_factors}")
    
    return X_genes, X_proteins

X_genes, X_proteins = generate_omics_data()

In [ ]:
# 2.2 Apply NMF to each modality
def apply_nmf_multiomics(X_genes, X_proteins, n_components=5):
    """Apply NMF to multi-omics data"""
    
    print("\n🔬 Applying NMF to Multi-omics Data")
    print("=" * 50)
    
    # NMF for gene expression
    nmf_genes = NMF(n_components=n_components, random_state=42, max_iter=500)
    W_genes = nmf_genes.fit_transform(X_genes)
    H_genes = nmf_genes.components_
    
    print(f"\n📊 Gene Expression Factorization:")
    print(f"  W (samples × factors): {W_genes.shape}")
    print(f"  H (factors × genes): {H_genes.shape}")
    print(f"  Reconstruction error: {nmf_genes.reconstruction_err_:.4f}")
    
    # NMF for protein abundance
    nmf_proteins = NMF(n_components=n_components, random_state=42, max_iter=500)
    W_proteins = nmf_proteins.fit_transform(X_proteins)
    H_proteins = nmf_proteins.components_
    
    print(f"\n📊 Protein Abundance Factorization:")
    print(f"  W (samples × factors): {W_proteins.shape}")
    print(f"  H (factors × proteins): {H_proteins.shape}")
    print(f"  Reconstruction error: {nmf_proteins.reconstruction_err_:.4f}")
    
    return W_genes, H_genes, W_proteins, H_proteins

W_g, H_g, W_p, H_p = apply_nmf_multiomics(X_genes, X_proteins)

In [ ]:
# 2.3 Identify shared factors across modalities
def identify_shared_factors(W_genes, W_proteins):
    """Find correlation between factors across modalities"""
    
    n_factors = W_genes.shape[1]
    correlation_matrix = np.zeros((n_factors, n_factors))
    
    for i in range(n_factors):
        for j in range(n_factors):
            correlation_matrix[i, j] = pearsonr(W_genes[:, i], W_proteins[:, j])[0]
    
    # Visualize correlation
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='RdBu_r', 
                center=0, vmin=-1, vmax=1,
                xticklabels=[f'Protein F{i+1}' for i in range(n_factors)],
                yticklabels=[f'Gene F{i+1}' for i in range(n_factors)])
    plt.title('Factor Correlation Between Gene Expression and Protein Abundance')
    plt.tight_layout()
    plt.show()
    
    # Find strongly correlated factor pairs (shared factors)
    print("\n🔍 Shared Factors (correlation > 0.5):")
    for i in range(n_factors):
        for j in range(n_factors):
            if abs(correlation_matrix[i, j]) > 0.5:
                print(f"  Gene Factor {i+1} ↔ Protein Factor {j+1}: r = {correlation_matrix[i, j]:.3f}")
    
    return correlation_matrix

corr_matrix = identify_shared_factors(W_g, W_p)

---
## Practice 3: Canonical Correlation Analysis (CCA)

### 🎯 Learning Objectives
- Implement CCA to find maximally correlated projections
- Understand canonical variates
- Compare with other integration methods

### 📖 Key Concepts
**CCA Objective**: Find $w_1$ and $w_2$ that maximize $\text{corr}(w_1^T X_1, w_2^T X_2)$

In [ ]:
# 3.1 Implement CCA
def perform_cca(X_mod1, X_mod2, n_components=3):
    """Perform Canonical Correlation Analysis"""
    
    print("\n🔗 Canonical Correlation Analysis")
    print("=" * 50)
    
    # Standardize data
    scaler1 = StandardScaler()
    scaler2 = StandardScaler()
    X1_scaled = scaler1.fit_transform(X_mod1)
    X2_scaled = scaler2.fit_transform(X_mod2)
    
    # Apply CCA
    cca = CCA(n_components=n_components)
    U, V = cca.fit_transform(X1_scaled, X2_scaled)
    
    print(f"\n📊 CCA Results:")
    print(f"  Canonical variates U (Modality 1): {U.shape}")
    print(f"  Canonical variates V (Modality 2): {V.shape}")
    
    # Calculate canonical correlations
    print(f"\n📈 Canonical Correlations:")
    for i in range(n_components):
        corr = pearsonr(U[:, i], V[:, i])[0]
        print(f"  Component {i+1}: r = {corr:.4f}")
    
    return cca, U, V

cca_model, U, V = perform_cca(X_genes, X_proteins)

In [ ]:
# 3.2 Visualize canonical variates
def visualize_cca(U, V):
    """Visualize the first two canonical variates"""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot of first canonical variate
    axes[0].scatter(U[:, 0], V[:, 0], alpha=0.6, s=50, c='#1E64C8')
    axes[0].set_xlabel('U₁ (Gene Expression)', fontsize=12)
    axes[0].set_ylabel('V₁ (Protein Abundance)', fontsize=12)
    axes[0].set_title('First Canonical Variate Pair', fontsize=13, fontweight='bold')
    corr1 = pearsonr(U[:, 0], V[:, 0])[0]
    axes[0].text(0.05, 0.95, f'r = {corr1:.4f}', transform=axes[0].transAxes,
                fontsize=11, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Scatter plot of second canonical variate
    axes[1].scatter(U[:, 1], V[:, 1], alpha=0.6, s=50, c='#5088d4')
    axes[1].set_xlabel('U₂ (Gene Expression)', fontsize=12)
    axes[1].set_ylabel('V₂ (Protein Abundance)', fontsize=12)
    axes[1].set_title('Second Canonical Variate Pair', fontsize=13, fontweight='bold')
    corr2 = pearsonr(U[:, 1], V[:, 1])[0]
    axes[1].text(0.05, 0.95, f'r = {corr2:.4f}', transform=axes[1].transAxes,
                fontsize=11, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ CCA successfully identifies correlated projections across modalities!")

visualize_cca(U, V)

---
## Practice 4: Similarity Network Fusion

### 🎯 Learning Objectives
- Build patient similarity networks from different data types
- Understand network-based integration
- Apply iterative fusion algorithm

### 📖 Key Concepts
**SNF**: Iteratively fuses multiple patient similarity networks to create an integrated network

In [ ]:
# 4.1 Create similarity networks
def create_similarity_network(X, k=5):
    """Create k-nearest neighbor similarity network"""
    
    # Compute pairwise distances
    distances = squareform(pdist(X, metric='euclidean'))
    
    # Convert to similarity (using Gaussian kernel)
    sigma = np.median(distances)
    similarity = np.exp(-distances**2 / (2 * sigma**2))
    
    # Keep only k-nearest neighbors
    n_samples = X.shape[0]
    W = np.zeros_like(similarity)
    
    for i in range(n_samples):
        # Find k nearest neighbors
        neighbors = np.argsort(distances[i])[1:k+1]  # Exclude self
        W[i, neighbors] = similarity[i, neighbors]
        W[neighbors, i] = similarity[neighbors, i]
    
    # Normalize
    row_sums = W.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # Avoid division by zero
    W = W / row_sums
    
    return W

def similarity_network_fusion(X_list, k=5, n_iterations=20):
    """Simple SNF implementation"""
    
    print("\n🌐 Similarity Network Fusion")
    print("=" * 50)
    
    # Create initial similarity networks
    networks = []
    for idx, X in enumerate(X_list):
        W = create_similarity_network(X, k=k)
        networks.append(W)
        print(f"  Network {idx+1} created: {W.shape}")
    
    # Simple fusion: average networks
    fused_network = np.mean(networks, axis=0)
    
    print(f"\n✅ Fused network created: {fused_network.shape}")
    
    return fused_network, networks

# Apply SNF
fused_net, individual_nets = similarity_network_fusion([X_genes, X_proteins])

In [ ]:
# 4.2 Clustering on fused network
def cluster_from_network(network, n_clusters=3):
    """Perform clustering using the fused network"""
    
    # Use network as affinity matrix for spectral clustering
    from sklearn.cluster import SpectralClustering
    
    clustering = SpectralClustering(n_clusters=n_clusters, 
                                   affinity='precomputed',
                                   random_state=42)
    labels = clustering.fit_predict(network)
    
    # Compute silhouette score
    # For spectral clustering with precomputed affinity, we use distance = 1 - affinity
    distance_matrix = 1 - network
    np.fill_diagonal(distance_matrix, 0)  # Distance to self is 0
    silhouette = silhouette_score(distance_matrix, labels, metric='precomputed')
    
    print(f"\n📊 Clustering Results:")
    print(f"  Number of clusters: {n_clusters}")
    print(f"  Cluster sizes: {np.bincount(labels)}")
    print(f"  Silhouette score: {silhouette:.4f}")
    
    return labels, silhouette

cluster_labels, sil_score = cluster_from_network(fused_net)

---
## Practice 5: Attention-based Integration

### 🎯 Learning Objectives
- Understand attention mechanism for multi-modal integration
- Implement a simple attention-based fusion
- Learn importance weighting of modalities

### 📖 Key Concepts
**Attention**: $\text{output} = \sum_i \alpha_i \cdot \text{feature}_i$ where $\alpha_i$ are learned attention weights

In [ ]:
# 5.1 Simple attention mechanism
def attention_fusion(features_list, method='mean'):
    """
    Simple attention-based fusion
    
    Parameters:
    - features_list: list of feature matrices
    - method: 'mean' or 'learned'
    """
    
    print("\n🎯 Attention-based Fusion")
    print("=" * 50)
    
    n_modalities = len(features_list)
    n_samples = features_list[0].shape[0]
    
    # Reduce each modality to same dimension using PCA
    n_components = 10
    reduced_features = []
    
    for idx, features in enumerate(features_list):
        pca = PCA(n_components=n_components)
        reduced = pca.fit_transform(features)
        reduced_features.append(reduced)
        print(f"  Modality {idx+1}: {features.shape} → {reduced.shape}")
    
    # Stack features
    stacked = np.stack(reduced_features, axis=1)  # (n_samples, n_modalities, n_components)
    
    if method == 'mean':
        # Equal attention weights
        attention_weights = np.ones(n_modalities) / n_modalities
        print(f"\n  Using equal attention weights: {attention_weights}")
    else:
        # Learn attention weights based on variance
        variances = [np.var(f, axis=0).mean() for f in reduced_features]
        attention_weights = np.array(variances) / sum(variances)
        print(f"\n  Learned attention weights: {attention_weights}")
    
    # Apply attention
    fused_features = np.sum(stacked * attention_weights[np.newaxis, :, np.newaxis], axis=1)
    
    print(f"\n✅ Fused features shape: {fused_features.shape}")
    
    return fused_features, attention_weights

fused_features, att_weights = attention_fusion([X_genes, X_proteins], method='learned')

In [ ]:
# 5.2 Visualize attention weights
def visualize_attention(attention_weights):
    """Visualize learned attention weights"""
    
    modality_names = ['Gene Expression', 'Protein Abundance']
    
    plt.figure(figsize=(8, 5))
    bars = plt.bar(modality_names, attention_weights, color=['#1E64C8', '#5088d4'])
    plt.ylabel('Attention Weight', fontsize=12)
    plt.title('Learned Attention Weights for Each Modality', fontsize=13, fontweight='bold')
    plt.ylim([0, max(attention_weights) * 1.2])
    
    # Add value labels on bars
    for bar, weight in zip(bars, attention_weights):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{weight:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Interpretation:")
    max_idx = np.argmax(attention_weights)
    print(f"  {modality_names[max_idx]} receives the highest attention weight")
    print(f"  This suggests it contains more informative signals for the task")

visualize_attention(att_weights)

---
## Practice 6: MOFA Analysis

### 🎯 Learning Objectives
- Understand Multi-Omics Factor Analysis framework
- Decompose variance into shared and specific factors
- Interpret factor contributions

### 📖 Key Concepts
**MOFA**: Bayesian factor analysis that identifies shared and modality-specific variation

In [ ]:
# 6.1 Simplified MOFA-like analysis using NMF
def mofa_like_analysis(data_list, n_factors=5):
    """
    Simplified MOFA-like analysis
    In practice, use the actual MOFA package: https://github.com/bioFAM/MOFA2
    """
    
    print("\n🔬 MOFA-like Factor Analysis")
    print("=" * 50)
    
    # Concatenate data
    X_concat = np.concatenate(data_list, axis=1)
    print(f"  Concatenated data shape: {X_concat.shape}")
    
    # Apply NMF
    nmf = NMF(n_components=n_factors, random_state=42, max_iter=500)
    W = nmf.fit_transform(X_concat)  # Sample factors
    H = nmf.components_  # Feature factors
    
    print(f"\n📊 Factorization Results:")
    print(f"  Sample factors W: {W.shape}")
    print(f"  Feature factors H: {H.shape}")
    
    # Split H back into modality-specific components
    n_genes = data_list[0].shape[1]
    H_genes = H[:, :n_genes]
    H_proteins = H[:, n_genes:]
    
    # Calculate variance explained by each factor
    var_explained = []
    for i in range(n_factors):
        # Reconstruction using only factor i
        reconstruction = np.outer(W[:, i], H[i, :])
        var = np.var(reconstruction)
        var_explained.append(var)
    
    var_explained = np.array(var_explained)
    var_explained_pct = var_explained / var_explained.sum() * 100
    
    print(f"\n📈 Variance Explained by Each Factor:")
    for i, pct in enumerate(var_explained_pct):
        print(f"  Factor {i+1}: {pct:.2f}%")
    
    return W, H_genes, H_proteins, var_explained_pct

W_mofa, H_genes_mofa, H_proteins_mofa, var_exp = mofa_like_analysis([X_genes, X_proteins])

In [ ]:
# 6.2 Visualize factor contributions
def visualize_mofa_results(W, var_explained_pct):
    """Visualize MOFA factor analysis results"""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Variance explained
    n_factors = len(var_explained_pct)
    axes[0].bar(range(1, n_factors+1), var_explained_pct, color='#1E64C8')
    axes[0].set_xlabel('Factor', fontsize=12)
    axes[0].set_ylabel('Variance Explained (%)', fontsize=12)
    axes[0].set_title('Variance Decomposition', fontsize=13, fontweight='bold')
    axes[0].set_xticks(range(1, n_factors+1))
    
    # Sample representation in factor space
    scatter = axes[1].scatter(W[:, 0], W[:, 1], c=np.arange(len(W)), 
                             cmap='viridis', s=50, alpha=0.6)
    axes[1].set_xlabel(f'Factor 1 ({var_explained_pct[0]:.1f}%)', fontsize=12)
    axes[1].set_ylabel(f'Factor 2 ({var_explained_pct[1]:.1f}%)', fontsize=12)
    axes[1].set_title('Samples in Factor Space', fontsize=13, fontweight='bold')
    plt.colorbar(scatter, ax=axes[1], label='Sample Index')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ MOFA reveals latent factors explaining variance across modalities!")

visualize_mofa_results(W_mofa, var_exp)

---
## Practice 7: Complete Integration Workflow

### 🎯 Learning Objectives
- Implement end-to-end integration pipeline
- Apply preprocessing, integration, and evaluation
- Compare multiple integration methods

### 📖 Workflow Steps
1. **Data Loading** → 2. **Preprocessing** → 3. **Integration** → 4. **Visualization** → 5. **Interpretation**

In [ ]:
# 7.1 Complete integration pipeline
def integration_pipeline(X_list, labels=None, method='pca'):
    """
    Complete multi-modal integration workflow
    
    Parameters:
    - X_list: list of data matrices
    - labels: ground truth labels (if available)
    - method: 'pca', 'nmf', or 'concat'
    """
    
    print("\n🔄 Complete Integration Pipeline")
    print("=" * 60)
    
    # Step 1: Preprocessing
    print("\n📝 Step 1: Preprocessing")
    X_scaled_list = []
    for idx, X in enumerate(X_list):
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled_list.append(X_scaled)
        print(f"  Modality {idx+1}: Standardized")
    
    # Step 2: Integration
    print(f"\n🔗 Step 2: Integration (method={method})")
    if method == 'concat':
        X_integrated = np.concatenate(X_scaled_list, axis=1)
    elif method == 'pca':
        X_concat = np.concatenate(X_scaled_list, axis=1)
        pca = PCA(n_components=10)
        X_integrated = pca.fit_transform(X_concat)
    elif method == 'nmf':
        X_concat = np.concatenate([np.abs(X) for X in X_scaled_list], axis=1)
        nmf = NMF(n_components=10, random_state=42)
        X_integrated = nmf.fit_transform(X_concat)
    
    print(f"  Integrated data shape: {X_integrated.shape}")
    
    # Step 3: Clustering
    print("\n📊 Step 3: Clustering")
    n_clusters = 3
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    pred_labels = kmeans.fit_predict(X_integrated)
    print(f"  Cluster sizes: {np.bincount(pred_labels)}")
    
    # Step 4: Evaluation
    print("\n✅ Step 4: Evaluation")
    silhouette = silhouette_score(X_integrated, pred_labels)
    print(f"  Silhouette score: {silhouette:.4f}")
    
    if labels is not None:
        ari = adjusted_rand_score(labels, pred_labels)
        print(f"  Adjusted Rand Index: {ari:.4f}")
    
    return X_integrated, pred_labels, silhouette

# Run pipeline with different methods
print("\n" + "="*60)
print("Comparing Integration Methods")
print("="*60)

methods = ['concat', 'pca', 'nmf']
results = {}

for method in methods:
    X_int, labels_pred, sil = integration_pipeline([X_genes, X_proteins], method=method)
    results[method] = {'silhouette': sil, 'integrated': X_int, 'labels': labels_pred}
    print("\n" + "-"*60)

---
## Practice 8: Visualization and Interpretation

### 🎯 Learning Objectives
- Visualize integrated data in low-dimensional space
- Compare different integration methods
- Interpret biological meaning of clusters

In [ ]:
# 8.1 Comprehensive visualization
def visualize_integration_results(results_dict):
    """Visualize and compare integration results"""
    
    n_methods = len(results_dict)
    fig, axes = plt.subplots(1, n_methods, figsize=(6*n_methods, 5))
    
    if n_methods == 1:
        axes = [axes]
    
    for idx, (method, result) in enumerate(results_dict.items()):
        X_int = result['integrated']
        labels = result['labels']
        
        # Use first two dimensions for visualization
        if X_int.shape[1] >= 2:
            x, y = X_int[:, 0], X_int[:, 1]
        else:
            x = X_int[:, 0]
            y = np.zeros_like(x)
        
        scatter = axes[idx].scatter(x, y, c=labels, cmap='Set2', s=50, alpha=0.7)
        axes[idx].set_xlabel('Component 1', fontsize=11)
        axes[idx].set_ylabel('Component 2', fontsize=11)
        axes[idx].set_title(f'{method.upper()}\n(Silhouette: {result["silhouette"]:.3f})',
                           fontsize=12, fontweight='bold')
        
        # Add cluster centers
        for cluster_id in np.unique(labels):
            mask = labels == cluster_id
            center_x = x[mask].mean()
            center_y = y[mask].mean()
            axes[idx].plot(center_x, center_y, 'k*', markersize=20, 
                          markeredgecolor='white', markeredgewidth=2)
    
    plt.tight_layout()
    plt.show()
    
    # Print comparison
    print("\n📊 Method Comparison Summary:")
    print("-" * 50)
    for method, result in results_dict.items():
        print(f"  {method.upper():10s} - Silhouette: {result['silhouette']:.4f}")
    
    # Find best method
    best_method = max(results_dict.items(), key=lambda x: x[1]['silhouette'])[0]
    print(f"\n✅ Best method: {best_method.upper()}")

visualize_integration_results(results)

In [ ]:
# 8.2 Final summary and comparison
def create_comparison_table(results_dict):
    """Create a comprehensive comparison table"""
    
    comparison_data = []
    for method, result in results_dict.items():
        comparison_data.append({
            'Method': method.upper(),
            'Silhouette Score': f"{result['silhouette']:.4f}",
            'Dimensions': result['integrated'].shape[1],
            'N Clusters': len(np.unique(result['labels']))
        })
    
    df = pd.DataFrame(comparison_data)
    
    print("\n" + "="*60)
    print("FINAL COMPARISON TABLE")
    print("="*60)
    print(df.to_string(index=False))
    print("="*60)
    
    return df

comparison_df = create_comparison_table(results)

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **Early vs Late Fusion**: Different strategies for combining multi-modal data
2. **Matrix Factorization (NMF)**: Extracting shared and specific factors from multi-omics data
3. **Canonical Correlation Analysis (CCA)**: Finding maximally correlated projections
4. **Similarity Network Fusion**: Network-based integration approach
5. **Attention Mechanisms**: Learning importance weights for different modalities
6. **MOFA**: Multi-omics factor analysis for variance decomposition
7. **Complete Pipeline**: End-to-end integration workflow
8. **Visualization**: Comparing and interpreting integration results

### Key Insights:

✅ **Integration Methods**:
- Early fusion captures feature interactions but can be computationally expensive
- Late fusion is more flexible and modular
- Matrix factorization reveals latent structure
- Network-based methods leverage similarity relationships

✅ **Practical Considerations**:
- Data preprocessing (standardization) is crucial
- Choice of method depends on data characteristics and research questions
- Visualization helps interpret biological meaning
- Multiple evaluation metrics provide comprehensive assessment

✅ **Applications**:
- Disease subtyping (identifying molecular subtypes)
- Biomarker discovery (finding multi-modal signatures)
- Treatment response prediction
- Systems medicine approaches

### Next Steps:

1. **Advanced Methods**:
   - Deep learning fusion (autoencoders, transformers)
   - Graph neural networks for network integration
   - Multimodal variational autoencoders

2. **Real Data Applications**:
   - TCGA pan-cancer multi-omics datasets
   - Spatial transcriptomics + imaging
   - Clinical + molecular data integration

3. **Biological Interpretation**:
   - Pathway enrichment analysis
   - Network analysis and disease modules
   - Clinical validation

### Additional Resources:

- **MOFA Package**: https://github.com/bioFAM/MOFA2
- **SNFtool**: https://github.com/maxconway/SNFtool
- **scikit-learn**: https://scikit-learn.org/
- **Review Papers**: Subramanian et al. (2020) "Multi-omics data integration"

---

## 🎓 Congratulations!

You've completed the hands-on practice for Lecture 12: Multi-Modal Data Integration!

**Key Takeaway**: Multi-modal data integration is essential for comprehensive understanding of biological systems and precision medicine. Different integration methods have different strengths - the choice depends on your specific research question and data characteristics.